In [15]:
import pymongo
from pymongo import MongoClient
from imbox import Imbox
import os
import traceback
import gridfs
import mysql.connector

In [16]:
connection = MongoClient('localhost',27017)
db = connection.dsWeek2
mysql_db = mysql.connector.connect(
    host="localhost",
    user = "root",
    passwd=""
)
mycursor = mysql_db.cursor()
mycursor.execute("use week2ds")
invocieCollection = db.invoice 

In [22]:
with Imbox('imap.gmail.com',
        username='dsweek2@gmail.com',
        password='passpass@123',
        ssl=True,
        ssl_context=None,
        starttls=False) as imbox:
    kaushalMessages = imbox.messages(sent_from='kaushal.binjola17@gmail.com')
    
    
    for uid, message in kaushalMessages:
        invoiceDetailStr = message.body['plain'][0]
        invoiceDetailStr = invoiceDetailStr.split('\r\n')
        invoiceDetails = list(filter(lambda x: len(x) > 0, invoiceDetailStr))
        attachment = message.attachments
        
        obj ={}
        obj["company"] = invoiceDetails[0]
        obj["dateOrdered"] = invoiceDetails[1][invoiceDetails[1].index(":")+2: ]
        obj["invoiceNum"] = invoiceDetails[2][invoiceDetails[2].index(":")+2: ]
        obj["itemOrdered"] = invoiceDetails[3][invoiceDetails[3].index(":")+2: ]
        obj["billingAddr"] = invoiceDetails[4][invoiceDetails[4].index(":")+2: ]
        
        mycursor.execute(f"""insert into invoice_info 
                        values('{obj["company"]}','{obj["dateOrdered"]}','{obj["invoiceNum"]}','{obj["itemOrdered"]}','{obj["billingAddr"]}')
                        """)
        
        mysql_db.commit()
        
        for idx, attachment in enumerate(message.attachments):
            try:
                att_fn = attachment.get('filename')
                download_path = f"./attachments/{att_fn}"
                with open(download_path, "wb") as fp:
                    fp.write(attachment.get('content').read())
                    
                fileData = open(download_path,"rb")
                data = fileData.read()
                fs = gridfs.GridFS(db)
                fs.put(data, filename=att_fn)
            except:
                print(traceback.print_exc())
        
        